In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from PIL import Image
from skimage.transform import resize
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_y')
test_fov = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_fov')
train_x, train_y, test_x, test_y, test_fov

(dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512), dtype=float32, chunksize=(20, 512, 512)>)

In [3]:
def get_inside_fov(test_fov):
    flat = test_fov.ravel()
    flat = da.where(flat>0.5, 1, 0)
    index_inside_fov = da.where(flat)[0]
    return index_inside_fov
inside_fov = get_inside_fov(test_fov).compute()
inside_fov

array([  14069,   14070,   14071, ..., 5235971, 5235972, 5235973])

In [5]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters=24, dropout=0.35):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 8, 8, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 8, 8, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 8, 8, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 8, 8, 128)
    drop_5 = tf.keras.layers.SpatialDropout2D(dropout)(conv_12)#(-1, 8, 8, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_5) #(-1, 16, 16, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 16, 16, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 16, 16, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 16, 16, 64)
    drop_6 = tf.keras.layers.SpatialDropout2D(dropout)(conv_14) #(-1, 16, 16, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_6) #(-1, 32, 32, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 32, 32, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 32, 32, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 32, 32, 32)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16) #(-1, 32, 32, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 64, 64, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 64, 64, 48)
    conv_17 = build_block(upsp_4, n_filters * 2)#(-1, 64, 64, 16)
    conv_18 = build_block(conv_17, n_filters * 2)#(-1, 64, 64, 16)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)#(-1, 64, 64, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

def random_crop(img, msk, random_crop_size):
    # Note: image_data_format is 'channel_last'
    #assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :], msk[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_x_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        batch_y_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        for i in range(batch_x.shape[0]):
            batch_x_crops[i], batch_y_crops[i] = random_crop(batch_x[i], batch_y[i], (crop_length, crop_length))
        yield (batch_x_crops, batch_y_crops)
        
class roc_callback(tf.keras.callbacks.Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        bin_test_y = np.where(self.y_val.compute()>0.5, 1, 0)
        y_pred_val = self.model.predict(self.x_val, batch_size=1)
        roc_val = roc_auc_score(bin_test_y.ravel()[inside_fov], y_pred_val.ravel()[inside_fov])
        print('aucroc = ', roc_val)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
        
model = build_unet(input_shape=(None, None, 1))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, None, None, 2 216         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, None, None, 2 96          conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, None, None, 2 0           batch_normalization_18[0][0]     
__________________________________________________________________________________________________
conv2d_20 

In [ ]:
epochs, batch_size, lr, filepath = 1000000, 4, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True) #width_shift_range=0.1, height_shift_range=0.1, 
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = crop_generator(zip(image_generator, mask_generator), 64)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())
auc_roc = roc_callback(training_data=(train_x, train_y),validation_data=(test_x, test_y))

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(print(model.metrics_names))

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=0,
                    callbacks=[checkpoint, tb, auc_roc])

['loss', 'acc']
None
Epoch 1/1000000
4/5 [=======================>......] - ETA: 2s - loss: 3.8576 - acc: 0.4052
Epoch 00001: val_acc improved from -inf to 0.22216, saving model to day_1.weights.best.hdf5
aucroc =  0.6099956830427465
5/5 [==============================] - 17s 3s/step - loss: 3.6431 - acc: 0.4406 - val_loss: 0.7178 - val_acc: 0.2222
Epoch 2/1000000
4/5 [=======================>......] - ETA: 0s - loss: 3.1571 - acc: 0.4610
Epoch 00002: val_acc improved from 0.22216 to 0.85462, saving model to day_1.weights.best.hdf5
aucroc =  0.4317101136573449
5/5 [==============================] - 6s 1s/step - loss: 3.0706 - acc: 0.4801 - val_loss: 0.5722 - val_acc: 0.8546
Epoch 3/1000000
4/5 [=======================>......] - ETA: 0s - loss: 2.7555 - acc: 0.5730
Epoch 00003: val_acc did not improve from 0.85462
aucroc =  0.4405186543120755
5/5 [==============================] - 5s 907ms/step - loss: 2.7632 - acc: 0.5810 - val_loss: 1.6096 - val_acc: 0.6791
Epoch 4/1000000
4/5 [======

4/5 [=======================>......] - ETA: 0s - loss: 1.7742 - acc: 0.8053
Epoch 00028: val_acc did not improve from 0.87861
aucroc =  0.543606698470162
5/5 [==============================] - 5s 941ms/step - loss: 1.6912 - acc: 0.8271 - val_loss: 0.3116 - val_acc: 0.8786
Epoch 29/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.8095 - acc: 0.7765
Epoch 00029: val_acc did not improve from 0.87861
aucroc =  0.5705886347708257
5/5 [==============================] - 5s 937ms/step - loss: 1.8269 - acc: 0.7661 - val_loss: 0.3179 - val_acc: 0.8783
Epoch 30/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.6480 - acc: 0.8404
Epoch 00030: val_acc did not improve from 0.87861
aucroc =  0.5866214201125277
5/5 [==============================] - 5s 934ms/step - loss: 1.5512 - acc: 0.8570 - val_loss: 0.3320 - val_acc: 0.8769
Epoch 31/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.5477 - acc: 0.8190
Epoch 00031: val_acc did not improve from 0.87861
au

Epoch 56/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.3304 - acc: 0.8410
Epoch 00056: val_acc did not improve from 0.87861
aucroc =  0.6324179495242498
5/5 [==============================] - 5s 944ms/step - loss: 1.3819 - acc: 0.8339 - val_loss: 0.2941 - val_acc: 0.8786
Epoch 57/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.5060 - acc: 0.8392
Epoch 00057: val_acc did not improve from 0.87861
aucroc =  0.6735061561740957
5/5 [==============================] - 5s 944ms/step - loss: 1.4545 - acc: 0.8419 - val_loss: 0.2976 - val_acc: 0.8786
Epoch 58/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.3087 - acc: 0.8335
Epoch 00058: val_acc did not improve from 0.87861
aucroc =  0.6515425860161997
5/5 [==============================] - 5s 942ms/step - loss: 1.2943 - acc: 0.8355 - val_loss: 0.2890 - val_acc: 0.8786
Epoch 59/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.3407 - acc: 0.8565
Epoch 00059: val_acc improved from

Epoch 84/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.1380 - acc: 0.8769
Epoch 00084: val_acc did not improve from 0.87927
aucroc =  0.6222462021789563
5/5 [==============================] - 5s 945ms/step - loss: 1.1717 - acc: 0.8712 - val_loss: 0.3385 - val_acc: 0.8782
Epoch 85/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.1828 - acc: 0.8793
Epoch 00085: val_acc did not improve from 0.87927
aucroc =  0.6262888224077036
5/5 [==============================] - 5s 945ms/step - loss: 1.1404 - acc: 0.8851 - val_loss: 0.3096 - val_acc: 0.8784
Epoch 86/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.3429 - acc: 0.8310
Epoch 00086: val_acc did not improve from 0.87927
aucroc =  0.6477098290773632
5/5 [==============================] - 5s 942ms/step - loss: 1.2530 - acc: 0.8445 - val_loss: 0.3043 - val_acc: 0.8785
Epoch 87/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.9729 - acc: 0.8811
Epoch 00087: val_acc did not impro

Epoch 112/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.6955 - acc: 0.7743
Epoch 00112: val_acc did not improve from 0.89069
aucroc =  0.6383118583279986
5/5 [==============================] - 5s 941ms/step - loss: 1.6032 - acc: 0.7911 - val_loss: 0.3454 - val_acc: 0.8795
Epoch 113/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.1072 - acc: 0.8561
Epoch 00113: val_acc did not improve from 0.89069
aucroc =  0.6677771237968929
5/5 [==============================] - 5s 943ms/step - loss: 1.1016 - acc: 0.8567 - val_loss: 0.4501 - val_acc: 0.8788
Epoch 114/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.3237 - acc: 0.8542
Epoch 00114: val_acc did not improve from 0.89069
aucroc =  0.6597740664424578
5/5 [==============================] - 5s 938ms/step - loss: 1.2280 - acc: 0.8610 - val_loss: 0.4245 - val_acc: 0.8790
Epoch 115/1000000
4/5 [=======================>......] - ETA: 0s - loss: 1.1062 - acc: 0.8587
Epoch 00115: val_acc did not i

...

4/5 [=======================>......] - ETA: 0s - loss: 0.4109 - acc: 0.9130
Epoch 09133: val_acc did not improve from 0.92446
aucroc =  0.975172542954262
5/5 [==============================] - 5s 1s/step - loss: 0.3980 - acc: 0.9159 - val_loss: 0.0900 - val_acc: 0.9237
Epoch 9134/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4170 - acc: 0.9103
Epoch 09134: val_acc did not improve from 0.92446
aucroc =  0.974963853090345
5/5 [==============================] - 5s 1s/step - loss: 0.4331 - acc: 0.9055 - val_loss: 0.0891 - val_acc: 0.9238
Epoch 9135/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5653 - acc: 0.8740
Epoch 09135: val_acc did not improve from 0.92446
aucroc =  0.9751499140691257
5/5 [==============================] - 5s 1s/step - loss: 0.5435 - acc: 0.8778 - val_loss: 0.0894 - val_acc: 0.9238
Epoch 9136/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5664 - acc: 0.8893
Epoch 09136: val_acc did not improve from 0.92446
aucroc

aucroc =  0.9764498658256546
5/5 [==============================] - 5s 1s/step - loss: 0.4636 - acc: 0.8993 - val_loss: 0.0977 - val_acc: 0.9208
Epoch 9162/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3690 - acc: 0.9192
Epoch 09162: val_acc did not improve from 0.92446
aucroc =  0.9766118687298466
5/5 [==============================] - 5s 1s/step - loss: 0.3912 - acc: 0.9148 - val_loss: 0.0976 - val_acc: 0.9208
Epoch 9163/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5508 - acc: 0.8854
Epoch 09163: val_acc did not improve from 0.92446
aucroc =  0.9766938300483048
5/5 [==============================] - 5s 1s/step - loss: 0.5354 - acc: 0.8870 - val_loss: 0.0961 - val_acc: 0.9212
Epoch 9164/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5533 - acc: 0.8829
Epoch 09164: val_acc did not improve from 0.92446
aucroc =  0.9761527215238687
5/5 [==============================] - 5s 1s/step - loss: 0.5648 - acc: 0.8793 - val_loss: 0.0986 - v

Epoch 9190/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4710 - acc: 0.9017
Epoch 09190: val_acc did not improve from 0.92446
aucroc =  0.972840497631987
5/5 [==============================] - 5s 1s/step - loss: 0.4388 - acc: 0.9069 - val_loss: 0.0942 - val_acc: 0.9237
Epoch 9191/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4307 - acc: 0.9060
Epoch 09191: val_acc did not improve from 0.92446
aucroc =  0.9725182495835429
5/5 [==============================] - 5s 1s/step - loss: 0.4555 - acc: 0.9024 - val_loss: 0.0913 - val_acc: 0.9239
Epoch 9192/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4866 - acc: 0.9021
Epoch 09192: val_acc did not improve from 0.92446
aucroc =  0.9758080743636732
5/5 [==============================] - 5s 1s/step - loss: 0.4704 - acc: 0.9034 - val_loss: 0.0891 - val_acc: 0.9240
Epoch 9193/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.2937 - acc: 0.9409
Epoch 09193: val_acc did not improve

4/5 [=======================>......] - ETA: 0s - loss: 0.4247 - acc: 0.9067
Epoch 09218: val_acc did not improve from 0.92446
aucroc =  0.975647248512231
5/5 [==============================] - 5s 993ms/step - loss: 0.4158 - acc: 0.9089 - val_loss: 0.0939 - val_acc: 0.9215
Epoch 9219/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4629 - acc: 0.9027
Epoch 09219: val_acc did not improve from 0.92446
aucroc =  0.9755653826492111
5/5 [==============================] - 5s 1s/step - loss: 0.4715 - acc: 0.8986 - val_loss: 0.0960 - val_acc: 0.9215
Epoch 9220/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4018 - acc: 0.9157
Epoch 09220: val_acc did not improve from 0.92446
aucroc =  0.9743399574066619
5/5 [==============================] - 5s 995ms/step - loss: 0.4311 - acc: 0.9052 - val_loss: 0.0996 - val_acc: 0.9213
Epoch 9221/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5212 - acc: 0.8958
Epoch 09221: val_acc did not improve from 0.92446

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

